<a href="https://colab.research.google.com/github/tywellik/covid-education-impacts/blob/main/COVID_Education_Emma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# read in products
products = pd.read_csv('gdrive/MyDrive/data/products_info.csv')
product_ids = products['LP ID'].to_list()

In [4]:
products.head()

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC - Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC - Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,"LC - Sites, Resources & Reference - Games & Si..."
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC - Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC - Digital Learning Platforms


In [5]:
engagement = pd.DataFrame()
directory = 'gdrive/MyDrive/data/engagement_data/'
for filename in os.listdir(directory):
    district_id = filename.split('.')[0]
    dist_df = pd.read_csv(directory + filename)
    dist_df['district_id'] = district_id
    engagement=pd.concat([engagement, dist_df], axis=0)

engagement.head()

,time,lp_id,pct_access,engagement_index,district_id
0,2020-01-01,93690.0,0.00,NaN,1000
1,2020-01-01,17941.0,0.03,0.90,1000
2,2020-01-01,65358.0,0.03,1.20,1000
3,2020-01-01,98265.0,0.57,37.79,1000
4,2020-01-01,59257.0,0.00,NaN,1000


In [6]:
# update datatypes
engagement['time'] = pd.to_datetime(engagement['time'])
engagement['district_id'] = engagement['district_id'].astype(int)

In [7]:
# get day of week for each date and filter out weekends
engagement['dayOfWeek'] = engagement['time'].dt.day_name()
engagement = engagement[engagement['dayOfWeek'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]
del engagement['dayOfWeek']

In [8]:
# also filter out summer and winter breaks
engagement = engagement[engagement['time'] >= pd.to_datetime('2020-01-06')]
engagement = engagement[(engagement['time'] <= pd.to_datetime('2020-06-05')) | (engagement['time'] >= pd.to_datetime('2020-08-17'))]
engagement = engagement[engagement['time'] <= pd.to_datetime('2020-12-18')]

In [9]:
engagement.head()

,time,lp_id,pct_access,engagement_index,district_id
1095,2020-01-06,34201.0,0.00,NaN,1000
1096,2020-01-06,28786.0,0.03,0.30,1000
1097,2020-01-06,33624.0,0.57,28.13,1000
1098,2020-01-06,25559.0,4.07,606.22,1000
1099,2020-01-06,73814.0,0.00,NaN,1000


In [10]:
# filter engagement for product ids in the product df that are classroom management
products_CM = products[products['Primary Essential Function'].str.contains("CM - ", na = False)]
products_CM.head()

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
11,72703,http://betterlesson.com/,Better Lesson,BetterLesson,PreK-12; Higher Ed,CM - Teacher Resources - Professional Learning
19,47267,http://www.classdojo.com/,ClassDojo,"ClassDojo, Inc.",PreK-12,CM - Classroom Engagement & Instruction - Comm...
43,23927,https://goguardian.com/,GoGuardian,GoGuardian,PreK-12,CM - Classroom Engagement & Instruction - Clas...
44,86703,https://www.gonoodle.com/,GoNoodle,HealthTeacher,PreK-12,CM - Classroom Engagement & Instruction - Clas...
51,33185,http://www.google.com/forms/about/,Google Forms,Google LLC,PreK-12; Higher Ed; Corporate,CM - Classroom Engagement & Instruction - Asse...


In [11]:
#Filter out so only CM platforms are included 
product_ids = products_CM['LP ID'].to_list()
engagement = engagement[engagement['lp_id'].isin(product_ids)]

In [12]:
engagement.head()

,time,lp_id,pct_access,engagement_index,district_id
1120,2020-01-06,59304.0,0.00,NaN,1000
1132,2020-01-06,33185.0,11.16,962.30,1000
1166,2020-01-06,86703.0,0.06,8.08,1000
1167,2020-01-06,61945.0,0.00,NaN,1000
1273,2020-01-06,21917.0,0.00,NaN,1000


In [13]:
# pivot engagement data
# pivot engagement_index
engagement_index = pd.pivot_table(engagement, values='engagement_index', index = ['district_id'], columns = ['lp_id'], aggfunc = 'mean')
engagement_index.columns = engagement_index.columns.astype(int).astype(str)
engagement_index = engagement_index.add_prefix('eng_ind_')
engagement_index.reset_index(drop = False, inplace = True)
engagement_index.fillna(0, inplace = True)
# pivot pct_access
engagement_access = pd.pivot_table(engagement, values='pct_access', index = ['district_id'], columns = ['lp_id'], aggfunc = 'mean')
engagement_access.columns = engagement_access.columns.astype(int).astype(str)
engagement_access = engagement_access.add_prefix('pct_acc_')
engagement_access.reset_index(drop = False, inplace = True)
engagement_access.fillna(0, inplace = True)
# merge pivoted engagement tables
engagement = pd.merge(left = engagement_index, right = engagement_access, on = 'district_id', how = 'outer')
engagement = engagement.groupby('district_id').mean()
engagement.head()

lp_id,eng_ind_13945,eng_ind_18982,eng_ind_20619,eng_ind_21917,eng_ind_22767,eng_ind_23927,eng_ind_26696,eng_ind_30008,eng_ind_32032,eng_ind_32050,eng_ind_33185,eng_ind_47267,eng_ind_48333,eng_ind_49094,eng_ind_50282,eng_ind_50626,eng_ind_50692,eng_ind_52448,eng_ind_55032,eng_ind_59304,eng_ind_61945,eng_ind_64850,eng_ind_65277,eng_ind_69706,eng_ind_72703,eng_ind_73323,eng_ind_78662,eng_ind_86703,eng_ind_93376,eng_ind_94218,eng_ind_95767,eng_ind_95859,eng_ind_98845,eng_ind_99580,pct_acc_13945,pct_acc_18982,pct_acc_20619,pct_acc_21917,pct_acc_22767,pct_acc_23927,pct_acc_26696,pct_acc_30008,pct_acc_32032,pct_acc_32050,pct_acc_33185,pct_acc_47267,pct_acc_48333,pct_acc_49094,pct_acc_50282,pct_acc_50626,pct_acc_50692,pct_acc_52448,pct_acc_55032,pct_acc_59304,pct_acc_61945,pct_acc_64850,pct_acc_65277,pct_acc_69706,pct_acc_72703,pct_acc_73323,pct_acc_78662,pct_acc_86703,pct_acc_93376,pct_acc_94218,pct_acc_95767,pct_acc_95859,pct_acc_98845,pct_acc_99580
district_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000,0.000,0.000000,0.300000,0.598500,0.602857,4.398581,3.178750,1.069167,0.000000,2.892500,1700.838359,6.906000,4.461875,10.488000,1.263000,0.284000,3.503129,0.391250,1.507857,0.925238,267.134430,7.541667,1.393208,0.0,4.781500,0.440000,1.056667,66.847214,0.260000,0.000000,0.290000,16.460891,4705.078592,0.000000,0.000000,0.000000,0.001000,0.009524,0.018108,0.104747,0.011242,0.003103,0.000000,0.005954,12.855750,0.065431,0.124080,0.128351,0.007424,0.003659,0.122563,0.007742,0.003358,0.005146,1.218602,0.005294,0.026450,0.0,0.010171,0.002000,0.006102,0.468947,0.001071,0.000000,0.030000,0.042258,17.851197,0.000000
1039,0.880,0.000000,6.658000,1.290000,1.530000,0.000000,4.877692,1.890000,0.000000,11.290896,525.107143,12.811562,31.984113,17.967143,15.449524,0.000000,0.000000,2.885385,3.090000,1.430000,597.034507,4.975000,4.761591,0.0,32.594000,1.983429,1.722500,21.981136,3.050000,0.000000,0.000000,124.484570,39.457931,1.150000,0.090000,0.000000,0.118000,0.085000,0.080000,0.000000,0.170000,0.081429,0.000000,0.220896,3.368857,0.280938,0.437742,0.301429,0.252222,0.000000,0.000000,0.133846,0.104000,0.087143,5.638028,0.080000,0.129205,0.0,0.136000,0.102857,0.075000,0.113636,0.080000,0.000000,0.000000,0.749301,0.472644,0.080000
1044,0.174,0.110476,0.406857,0.186901,1.499130,0.199211,5.321630,0.787594,0.113833,0.261829,1563.024800,5.655397,1.137244,2.667989,16.571250,0.225217,2.410497,0.204937,0.462841,0.744907,1806.681055,0.502745,2.300781,0.0,1.727470,0.998000,0.271910,36.245150,0.216875,0.104000,0.083077,9.441200,0.875714,1.831481,0.000000,0.000698,0.004928,0.001875,0.073871,0.000566,0.068316,0.011579,0.001846,0.001724,13.077000,0.076582,0.077176,0.059358,0.303692,0.002043,0.102031,0.010349,0.004324,0.014757,24.418600,0.001212,0.071179,0.0,0.003281,0.006045,0.003793,0.168650,0.000274,0.002024,0.001765,0.043450,0.054397,0.039659
1052,0.670,3.747917,1.706250,7.431333,0.647143,0.520000,1.422667,2.670000,0.356667,6.968571,1991.642300,12.229444,3.314324,2.222000,155.096337,0.732857,1.164211,0.776667,1.660714,1.669444,48.878065,0.786667,5.234497,0.0,11.358824,8.480000,1.137692,64.457246,0.340000,1.258889,0.925000,10.963770,15007.919930,13.344559,0.004286,0.022444,0.010690,0.007059,0.017586,0.020833,0.005385,0.001034,0.007333,0.005686,18.025400,0.071288,0.049274,0.044103,1.900056,0.002222,0.007161,0.007500,0.006121,0.030059,0.389620,0.001667,0.069585,0.0,0.010635,0.004000,0.013846,0.227244,0.000588,0.042708,0.016000,0.041053,39.399161,0.390805
1131,0.000,0.000000,0.000000,1.525000,0.390000,0.000000,0.757143,1.060000,0.000000,0.000000,423.615411,1.904000,6.721290,2.165000,2.048333,0.000000,0.000000,0.000000,0.000000,1.133750,7.822297,0.770000,5.302838,0.0,9.422000,0.686000,5.660000,7.172593,0.000000,0.783333,0.000000,9.382500,3.924750,0.000000,0.000000,0.000000,0.000000,0.011429,0.020000,0.000000,0.005079,0.006316,0.000000,0.000000,3.242262,0.003268,0.052317,0.006400,0.035263,0.000000,0.000000,0.000000,0.000000,0.005424,0.04602

In [14]:
# read in districts data
districts = pd.read_csv('gdrive/MyDrive/data/districts_info.csv')
del districts['county_connections_ratio']
# get values from the ranges
districts['pct_black/hispanic'] = districts['pct_black/hispanic'].str.strip('[]')
districts['pct_free/reduced'] = districts['pct_free/reduced'].str.strip('[]')
districts['pp_total_raw'] = districts['pp_total_raw'].str.strip('[]')
districts['pct_black/hispanic'] = districts['pct_black/hispanic'].str.split(', ', expand = True)[1]
districts['pct_free/reduced'] = districts['pct_free/reduced'].str.split(', ', expand = True)[1]
districts['pp_total_raw'] = districts['pp_total_raw'].str.split(', ', expand = True)[1]
print('districts')
print(districts)

districts
     district_id       state  ... pct_free/reduced pp_total_raw
0           8815    Illinois  ...              0.2        16000
1           2685         NaN  ...              NaN          NaN
2           4921        Utah  ...              0.4         8000
3           3188         NaN  ...              NaN          NaN
4           2238         NaN  ...              NaN          NaN
..           ...         ...  ...              ...          ...
228         9515    New York  ...              0.6        20000
229         8103   Tennessee  ...              NaN        10000
230         4929    Virginia  ...              0.6        14000
231         7975  California  ...              0.8          NaN
232         7164  California  ...              0.8          NaN

[233 rows x 6 columns]


In [15]:
# merge districts and engagement table
engagement = pd.merge(left = districts, right = engagement, on = 'district_id', how = 'outer')

In [16]:
engagement.drop(columns = ['district_id', 'locale'], inplace = True)

In [17]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 0 to 232
Data columns (total 72 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   state               176 non-null    object 
 1   pct_black/hispanic  176 non-null    object 
 2   pct_free/reduced    148 non-null    object 
 3   pp_total_raw        118 non-null    object 
 4   eng_ind_13945       233 non-null    float64
 5   eng_ind_18982       233 non-null    float64
 6   eng_ind_20619       233 non-null    float64
 7   eng_ind_21917       233 non-null    float64
 8   eng_ind_22767       233 non-null    float64
 9   eng_ind_23927       233 non-null    float64
 10  eng_ind_26696       233 non-null    float64
 11  eng_ind_30008       233 non-null    float64
 12  eng_ind_32032       233 non-null    float64
 13  eng_ind_32050       233 non-null    float64
 14  eng_ind_33185       233 non-null    float64
 15  eng_ind_47267       233 non-null    float64
 16  eng_ind_

In [18]:
engagement['pct_free/reduced'] = engagement['pct_free/reduced'].astype(str).astype(float)
engagement['pp_total_raw'] = engagement['pp_total_raw'].astype(str).astype(float)

engagement.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 0 to 232
Data columns (total 72 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   state               176 non-null    object 
 1   pct_black/hispanic  176 non-null    object 
 2   pct_free/reduced    148 non-null    float64
 3   pp_total_raw        118 non-null    float64
 4   eng_ind_13945       233 non-null    float64
 5   eng_ind_18982       233 non-null    float64
 6   eng_ind_20619       233 non-null    float64
 7   eng_ind_21917       233 non-null    float64
 8   eng_ind_22767       233 non-null    float64
 9   eng_ind_23927       233 non-null    float64
 10  eng_ind_26696       233 non-null    float64
 11  eng_ind_30008       233 non-null    float64
 12  eng_ind_32032       233 non-null    float64
 13  eng_ind_32050       233 non-null    float64
 14  eng_ind_33185       233 non-null    float64
 15  eng_ind_47267       233 non-null    float64
 16  eng_ind_

In [54]:
#Drop rows with NaN states
engagement_state = engagement[engagement['state'].notna()]

engagement_state

,state,pct_black/hispanic,pct_free/reduced,pp_total_raw,eng_ind_13945,eng_ind_18982,eng_ind_20619,eng_ind_21917,eng_ind_22767,eng_ind_23927,eng_ind_26696,eng_ind_30008,eng_ind_32032,eng_ind_32050,eng_ind_33185,eng_ind_47267,eng_ind_48333,eng_ind_49094,eng_ind_50282,eng_ind_50626,eng_ind_50692,eng_ind_52448,eng_ind_55032,eng_ind_59304,eng_ind_61945,eng_ind_64850,eng_ind_65277,eng_ind_69706,eng_ind_72703,eng_ind_73323,eng_ind_78662,eng_ind_86703,eng_ind_93376,eng_ind_94218,eng_ind_95767,eng_ind_95859,eng_ind_98845,eng_ind_99580,pct_acc_13945,pct_acc_18982,pct_acc_20619,pct_acc_21917,pct_acc_22767,pct_acc_23927,pct_acc_26696,pct_acc_30008,pct_acc_32032,pct_acc_32050,pct_acc_33185,pct_acc_47267,pct_acc_48333,pct_acc_49094,pct_acc_50282,pct_acc_50626,pct_acc_50692,pct_acc_52448,pct_acc_55032,pct_acc_59304,pct_acc_61945,pct_acc_64850,pct_acc_65277,pct_acc_69706,pct_acc_72703,pct_acc_73323,pct_acc_78662,pct_acc_86703,pct_acc_93376,pct_acc_94218,pct_acc_95767,pct_acc_95859,pct_acc_98845,pct_acc_99580
0,Illinois,0.2,0.2,16000.0,0.665000,3.675526,1.255385,0.603056,0.527647,0.601538,1.184684,1.292093,0.371111,1.955091,5600.470432,16.566619,40.871270,2.345200,24.282238,0.700000,4.448218,0.478696,1.058235,5.668659,66.734416,1.467273,9.173469,0.000000,6.507692,119.911987,1.461429,48.240584,0.175000,0.808000,0.190000,10.761789,19281.368889,1.237586,0.004444,0.015797,0.007872,0.006466,0.015962,0.014500,0.016015,0.020221,0.005158,0.016162,28.932407,0.191739,0.579453,0.060137,0.395034,0.004444,0.129296,0.014000,0.007212,0.044930,0.575909,0.003333,0.154783,0.000000,0.007093,1.113962,0.028608,0.476038,0.000755,0.003871,0.010000,0.064029,44.259931,0.022982
2,Utah,0.2,0.4,8000.0,0.046364,0.291136,0.068125,0.082708,0.124744,0.050000,0.329868,0.312600,0.052222,0.452105,631.270928,3.721525,64.302090,0.581382,5.082216,0.252022,0.040000,0.275663,0.212807,0.596981,129.004106,0.507045,3.340829,85.468984,1.278065,0.502326,0.225094,11.935444,0.104000,0.249444,0.097500,2.683082,70.315423,51.653333,0.000286,0.002911,0.000094,0.001788,0.005114,0.000417,0.007168,0.000741,0.000522,0.004722,5.396701,0.040741,0.597687,0.016585,0.157614,0.005736,0.000000,0.013368,0.001290,0.012418,1.961854,0.004348,0.065602,0.860410,0.001214,0.020347,0.001979,0.079266,0.000000,0.004474,0.006667,0.015585,0.569859,1.253222
5,Wisconsin,0.2,0.2,12000.0,3.178333,1.605250,4.959136,7.032162,4.230000,26.100993,3.412887,6.730263,0.802222,4.910763,1905.985854,3.965870,79.625303,3.435625,26.644959,2.178289,10.617925,2.729024,1.241304,1.679683,2482.448846,2.234375,6.159394,0.000000,36.527143,44.373462,1.178333,57.870534,1.176923,30.999861,0.000000,7.060517,1230.590000,17.882895,0.011053,0.026056,0.033594,0.019419,0.172821,0.224367,0.059375,0.030732,0.024808,0.050753,19.259085,0.016083,0.905682,0.098723,0.704557,0.032963,0.018864,0.058906,0.008723,0.023311,29.852562,0.010000,0.096500,0.000000,0.011000,0.388091,0.010286,0.215732,0.009038,1.018780,0.000000,0.037187,10.332535,0.500964
6,Utah,0.2,0.6,8000.0,0.000000,0.000000,0.000000,2.025000,3.560000,28.193125,2.876000,105.075769,0.000000,0.000000,251.299942,15.697692,218.882441,2.010000,0.000000,0.000000,2.010000,0.000000,4.060000,3.520000,288.069565,0.000000,5.380909,0.000000,1.680000,0.000000,0.000000,9.331538,0.000000,0.000000,0.000000,0.000000,217.376167,14.130000,0.000000,0.000000,0.000000,0.019048,0.062857,0.204178,0.035000,0.496279,0.000000,0.000000,2.210361,0.159778,1.574331,0.022222,0.000000,0.000000,0.200000,0.000000,0.040000,0.024828,3.314172,0.000000,0.018000,0.000000,0.042500,0.000000,0.000000,0.200714,0.000000,0.000000,0.000000,0.000000,1.916532,0.045000
7,North Carolina,0.4,0.4,10000.0,0.029167,0.104364,0.104717,0.077600,0.481852,0.090000,2.488881,0.431758,0.105432,0.152131,657.983800,3.658617,1.181583,2.964933,22.475436,1.160476,0.052500,0.185670,0.154026,0.276667,76.885179,0.431087,3.241209,1.360000,2.175098,9.553522,0.173529,6.432026,0.082667,6.718313,0.000000,3.101712,316.505786,69.016333,0.000000,0.000060,0.000323,0.000914,0.0

In [55]:
#Replace NAN values for pp_total_raw and pct_free/reduced with average for that state
engagement_state['pp_total_raw'] = engagement_state['pp_total_raw'].fillna(engagement_state.groupby('state')['pp_total_raw'].transform('mean'))
engagement_state['pp_total_raw'] = engagement_state['pp_total_raw'].fillna(engagement_state['pp_total_raw'].mean())

engagement_state['pct_free/reduced'] = engagement_state['pct_free/reduced'].fillna(engagement_state.groupby('state')['pct_free/reduced'].transform('mean'))
engagement_state['pct_free/reduced'] = engagement_state['pct_free/reduced'].fillna(engagement_state['pct_free/reduced'].mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [56]:
engagement_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 232
Data columns (total 72 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   state               176 non-null    object 
 1   pct_black/hispanic  176 non-null    object 
 2   pct_free/reduced    176 non-null    float64
 3   pp_total_raw        176 non-null    float64
 4   eng_ind_13945       176 non-null    float64
 5   eng_ind_18982       176 non-null    float64
 6   eng_ind_20619       176 non-null    float64
 7   eng_ind_21917       176 non-null    float64
 8   eng_ind_22767       176 non-null    float64
 9   eng_ind_23927       176 non-null    float64
 10  eng_ind_26696       176 non-null    float64
 11  eng_ind_30008       176 non-null    float64
 12  eng_ind_32032       176 non-null    float64
 13  eng_ind_32050       176 non-null    float64
 14  eng_ind_33185       176 non-null    float64
 15  eng_ind_47267       176 non-null    float64
 16  eng_ind_

In [57]:
engagement_state

,state,pct_black/hispanic,pct_free/reduced,pp_total_raw,eng_ind_13945,eng_ind_18982,eng_ind_20619,eng_ind_21917,eng_ind_22767,eng_ind_23927,eng_ind_26696,eng_ind_30008,eng_ind_32032,eng_ind_32050,eng_ind_33185,eng_ind_47267,eng_ind_48333,eng_ind_49094,eng_ind_50282,eng_ind_50626,eng_ind_50692,eng_ind_52448,eng_ind_55032,eng_ind_59304,eng_ind_61945,eng_ind_64850,eng_ind_65277,eng_ind_69706,eng_ind_72703,eng_ind_73323,eng_ind_78662,eng_ind_86703,eng_ind_93376,eng_ind_94218,eng_ind_95767,eng_ind_95859,eng_ind_98845,eng_ind_99580,pct_acc_13945,pct_acc_18982,pct_acc_20619,pct_acc_21917,pct_acc_22767,pct_acc_23927,pct_acc_26696,pct_acc_30008,pct_acc_32032,pct_acc_32050,pct_acc_33185,pct_acc_47267,pct_acc_48333,pct_acc_49094,pct_acc_50282,pct_acc_50626,pct_acc_50692,pct_acc_52448,pct_acc_55032,pct_acc_59304,pct_acc_61945,pct_acc_64850,pct_acc_65277,pct_acc_69706,pct_acc_72703,pct_acc_73323,pct_acc_78662,pct_acc_86703,pct_acc_93376,pct_acc_94218,pct_acc_95767,pct_acc_95859,pct_acc_98845,pct_acc_99580
0,Illinois,0.2,0.200000,16000.000000,0.665000,3.675526,1.255385,0.603056,0.527647,0.601538,1.184684,1.292093,0.371111,1.955091,5600.470432,16.566619,40.871270,2.345200,24.282238,0.700000,4.448218,0.478696,1.058235,5.668659,66.734416,1.467273,9.173469,0.000000,6.507692,119.911987,1.461429,48.240584,0.175000,0.808000,0.190000,10.761789,19281.368889,1.237586,0.004444,0.015797,0.007872,0.006466,0.015962,0.014500,0.016015,0.020221,0.005158,0.016162,28.932407,0.191739,0.579453,0.060137,0.395034,0.004444,0.129296,0.014000,0.007212,0.044930,0.575909,0.003333,0.154783,0.000000,0.007093,1.113962,0.028608,0.476038,0.000755,0.003871,0.010000,0.064029,44.259931,0.022982
2,Utah,0.2,0.400000,8000.000000,0.046364,0.291136,0.068125,0.082708,0.124744,0.050000,0.329868,0.312600,0.052222,0.452105,631.270928,3.721525,64.302090,0.581382,5.082216,0.252022,0.040000,0.275663,0.212807,0.596981,129.004106,0.507045,3.340829,85.468984,1.278065,0.502326,0.225094,11.935444,0.104000,0.249444,0.097500,2.683082,70.315423,51.653333,0.000286,0.002911,0.000094,0.001788,0.005114,0.000417,0.007168,0.000741,0.000522,0.004722,5.396701,0.040741,0.597687,0.016585,0.157614,0.005736,0.000000,0.013368,0.001290,0.012418,1.961854,0.004348,0.065602,0.860410,0.001214,0.020347,0.001979,0.079266,0.000000,0.004474,0.006667,0.015585,0.569859,1.253222
5,Wisconsin,0.2,0.200000,12000.000000,3.178333,1.605250,4.959136,7.032162,4.230000,26.100993,3.412887,6.730263,0.802222,4.910763,1905.985854,3.965870,79.625303,3.435625,26.644959,2.178289,10.617925,2.729024,1.241304,1.679683,2482.448846,2.234375,6.159394,0.000000,36.527143,44.373462,1.178333,57.870534,1.176923,30.999861,0.000000,7.060517,1230.590000,17.882895,0.011053,0.026056,0.033594,0.019419,0.172821,0.224367,0.059375,0.030732,0.024808,0.050753,19.259085,0.016083,0.905682,0.098723,0.704557,0.032963,0.018864,0.058906,0.008723,0.023311,29.852562,0.010000,0.096500,0.000000,0.011000,0.388091,0.010286,0.215732,0.009038,1.018780,0.000000,0.037187,10.332535,0.500964
6,Utah,0.2,0.600000,8000.000000,0.000000,0.000000,0.000000,2.025000,3.560000,28.193125,2.876000,105.075769,0.000000,0.000000,251.299942,15.697692,218.882441,2.010000,0.000000,0.000000,2.010000,0.000000,4.060000,3.520000,288.069565,0.000000,5.380909,0.000000,1.680000,0.000000,0.000000,9.331538,0.000000,0.000000,0.000000,0.000000,217.376167,14.130000,0.000000,0.000000,0.000000,0.019048,0.062857,0.204178,0.035000,0.496279,0.000000,0.000000,2.210361,0.159778,1.574331,0.022222,0.000000,0.000000,0.200000,0.000000,0.040000,0.024828,3.314172,0.000000,0.018000,0.000000,0.042500,0.000000,0.000000,0.200714,0.000000,0.000000,0.000000,0.000000,1.916532,0.045000
7,North Carolina,0.4,0.400000,10000.000000,0.029167,0.104364,0.104717,0.077600,0.481852,0.090000,2.488881,0.431758,0.105432,0.152131,657.983800,3.658617,1.181583,2.964933,22.475436,1.160476,0.052500,0.185670,0.154026,0.276667,76.885179,0.431087,3.241209,1.360000,2.175098,9.553522,0.173529,6.432026,0.082667,6.718313,0.000000,3.101712,316.505786

In [58]:
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.  Data originally from Wikipedia at the url:
# https://en.wikipedia.org/wiki/ISO_3166-2:US
#
# Automatically Generated 2021-09-11 18:04:36 via Jupyter Notebook from
# https://gist.github.com/rogerallen/d75440e8e5ea4762374dfd5c1ddf84e0 

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [59]:
engagement_state['State_Abb'] = engagement_state['state'].map(us_state_to_abbrev)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [60]:
import plotly.express as px  # Be sure to import express

fig = px.choropleth(engagement_state,  # Input Pandas DataFrame
                    locations="State_Abb",  # DataFrame column with locations
                    color="pp_total_raw",  # DataFrame column with color values
                    hover_name="state", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States
fig.update_layout(
    title_text = 'Per Pupil Expenditure By State', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()  # Output the plot to the screen

In [61]:
fig = px.choropleth(engagement_state,  # Input Pandas DataFrame
                    locations="State_Abb",  # DataFrame column with locations
                    color="pct_free/reduced",  # DataFrame column with color values
                    hover_name="state", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States
fig.update_layout(
    title_text = 'Percent Free/Reduced Lunch Per State', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()  # Output the plot to the screen

In [62]:
engagement_state.describe()

,pct_free/reduced,pp_total_raw,eng_ind_13945,eng_ind_18982,eng_ind_20619,eng_ind_21917,eng_ind_22767,eng_ind_23927,eng_ind_26696,eng_ind_30008,eng_ind_32032,eng_ind_32050,eng_ind_33185,eng_ind_47267,eng_ind_48333,eng_ind_49094,eng_ind_50282,eng_ind_50626,eng_ind_50692,eng_ind_52448,eng_ind_55032,eng_ind_59304,eng_ind_61945,eng_ind_64850,eng_ind_65277,eng_ind_69706,eng_ind_72703,eng_ind_73323,eng_ind_78662,eng_ind_86703,eng_ind_93376,eng_ind_94218,eng_ind_95767,eng_ind_95859,eng_ind_98845,eng_ind_99580,pct_acc_13945,pct_acc_18982,pct_acc_20619,pct_acc_21917,pct_acc_22767,pct_acc_23927,pct_acc_26696,pct_acc_30008,pct_acc_32032,pct_acc_32050,pct_acc_33185,pct_acc_47267,pct_acc_48333,pct_acc_49094,pct_acc_50282,pct_acc_50626,pct_acc_50692,pct_acc_52448,pct_acc_55032,pct_acc_59304,pct_acc_61945,pct_acc_64850,pct_acc_65277,pct_acc_69706,pct_acc_72703,pct_acc_73323,pct_acc_78662,pct_acc_86703,pct_acc_93376,pct_acc_94218,pct_acc_95767,pct_acc_95859,pct_acc_98845,pct_acc_99580
count,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000
mean,0.438121,13543.817527,2.753774,2.129820,2.351644,2.567462,2.142500,6.482855,8.962540,4.963765,0.642130,6.290973,1501.820479,13.915920,131.277593,8.175653,11.487763,2.831132,2.137815,2.042687,1.713402,2.422549,708.483836,4.690775,9.370485,14.722088,21.305991,24.766004,1.920616,38.180260,0.671444,6.526279,0.810584,16.599251,2996.906414,25.870332,0.007795,0.019704,0.020957,0.028389,0.074977,0.067398,0.096321,0.035120,0.012006,0.035452,11.043338,0.102649,0.862268,0.105230,0.177190,0.029694,0.040444,0.055578,0.013322,0.026898,7.684225,0.028304,0.097927,0.066291,0.025250,0.207518,0.020750,0.168177,0.008703,0.147833,0.019005,0.075521,10.488938,0.496364
std,0.194758,3751.377661,17.558640,3.507152,4.946297,3.605867,3.779017,17.606824,19.396614,12.429066,2.434260,14.426204,1375.699274,19.694979,294.170377,15.669817,22.379426,10.532118,5.118863,8.522856,2.969815,3.236241,1016.878909,10.501554,13.367117,120.105085,55.166785,56.340906,2.856814,43.140586,3.186074,17.579537,2.374620,19.527961,4853.689375,53.830287,0.023536,0.031466,0.042152,0.045160,0.109029,0.143068,0.180582,0.062043,0.035747,0.121768,7.672118,0.133324,1.422451,0.194044,0.261228,0.074650,0.110996,0.164030,0.032438,0.032504,11.165263,0.056279,0.077701,0.389886,0.068344,0.471246,0.031647,0.192149,0.048340,0.427110,0.039726,0.094140,12.488794,1.022396
min,0.200000,6000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.445517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.387963,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.413548,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007576,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,10000.000000,0.000000,0.295481,0.322232,0.416332,0.398947,0.285107,1.735331,0.586779,0.000000,1.360425,604.524546,3.724967,10.081029,1.330521,1.580085,0.280500,0.000000,0.283531,0.248800,0.638063,47.282358,0.554261,2.436245,0.000000,2.505000,0.422237,0.432430,12

In [63]:
engagement_state

,state,pct_black/hispanic,pct_free/reduced,pp_total_raw,eng_ind_13945,eng_ind_18982,eng_ind_20619,eng_ind_21917,eng_ind_22767,eng_ind_23927,eng_ind_26696,eng_ind_30008,eng_ind_32032,eng_ind_32050,eng_ind_33185,eng_ind_47267,eng_ind_48333,eng_ind_49094,eng_ind_50282,eng_ind_50626,eng_ind_50692,eng_ind_52448,eng_ind_55032,eng_ind_59304,eng_ind_61945,eng_ind_64850,eng_ind_65277,eng_ind_69706,eng_ind_72703,eng_ind_73323,eng_ind_78662,eng_ind_86703,eng_ind_93376,eng_ind_94218,eng_ind_95767,eng_ind_95859,eng_ind_98845,eng_ind_99580,pct_acc_13945,pct_acc_18982,pct_acc_20619,pct_acc_21917,pct_acc_22767,pct_acc_23927,pct_acc_26696,pct_acc_30008,pct_acc_32032,pct_acc_32050,pct_acc_33185,pct_acc_47267,pct_acc_48333,pct_acc_49094,pct_acc_50282,pct_acc_50626,pct_acc_50692,pct_acc_52448,pct_acc_55032,pct_acc_59304,pct_acc_61945,pct_acc_64850,pct_acc_65277,pct_acc_69706,pct_acc_72703,pct_acc_73323,pct_acc_78662,pct_acc_86703,pct_acc_93376,pct_acc_94218,pct_acc_95767,pct_acc_95859,pct_acc_98845,pct_acc_99580,State_Abb
0,Illinois,0.2,0.200000,16000.000000,0.665000,3.675526,1.255385,0.603056,0.527647,0.601538,1.184684,1.292093,0.371111,1.955091,5600.470432,16.566619,40.871270,2.345200,24.282238,0.700000,4.448218,0.478696,1.058235,5.668659,66.734416,1.467273,9.173469,0.000000,6.507692,119.911987,1.461429,48.240584,0.175000,0.808000,0.190000,10.761789,19281.368889,1.237586,0.004444,0.015797,0.007872,0.006466,0.015962,0.014500,0.016015,0.020221,0.005158,0.016162,28.932407,0.191739,0.579453,0.060137,0.395034,0.004444,0.129296,0.014000,0.007212,0.044930,0.575909,0.003333,0.154783,0.000000,0.007093,1.113962,0.028608,0.476038,0.000755,0.003871,0.010000,0.064029,44.259931,0.022982,IL
2,Utah,0.2,0.400000,8000.000000,0.046364,0.291136,0.068125,0.082708,0.124744,0.050000,0.329868,0.312600,0.052222,0.452105,631.270928,3.721525,64.302090,0.581382,5.082216,0.252022,0.040000,0.275663,0.212807,0.596981,129.004106,0.507045,3.340829,85.468984,1.278065,0.502326,0.225094,11.935444,0.104000,0.249444,0.097500,2.683082,70.315423,51.653333,0.000286,0.002911,0.000094,0.001788,0.005114,0.000417,0.007168,0.000741,0.000522,0.004722,5.396701,0.040741,0.597687,0.016585,0.157614,0.005736,0.000000,0.013368,0.001290,0.012418,1.961854,0.004348,0.065602,0.860410,0.001214,0.020347,0.001979,0.079266,0.000000,0.004474,0.006667,0.015585,0.569859,1.253222,UT
5,Wisconsin,0.2,0.200000,12000.000000,3.178333,1.605250,4.959136,7.032162,4.230000,26.100993,3.412887,6.730263,0.802222,4.910763,1905.985854,3.965870,79.625303,3.435625,26.644959,2.178289,10.617925,2.729024,1.241304,1.679683,2482.448846,2.234375,6.159394,0.000000,36.527143,44.373462,1.178333,57.870534,1.176923,30.999861,0.000000,7.060517,1230.590000,17.882895,0.011053,0.026056,0.033594,0.019419,0.172821,0.224367,0.059375,0.030732,0.024808,0.050753,19.259085,0.016083,0.905682,0.098723,0.704557,0.032963,0.018864,0.058906,0.008723,0.023311,29.852562,0.010000,0.096500,0.000000,0.011000,0.388091,0.010286,0.215732,0.009038,1.018780,0.000000,0.037187,10.332535,0.500964,WI
6,Utah,0.2,0.600000,8000.000000,0.000000,0.000000,0.000000,2.025000,3.560000,28.193125,2.876000,105.075769,0.000000,0.000000,251.299942,15.697692,218.882441,2.010000,0.000000,0.000000,2.010000,0.000000,4.060000,3.520000,288.069565,0.000000,5.380909,0.000000,1.680000,0.000000,0.000000,9.331538,0.000000,0.000000,0.000000,0.000000,217.376167,14.130000,0.000000,0.000000,0.000000,0.019048,0.062857,0.204178,0.035000,0.496279,0.000000,0.000000,2.210361,0.159778,1.574331,0.022222,0.000000,0.000000,0.200000,0.000000,0.040000,0.024828,3.314172,0.000000,0.018000,0.000000,0.042500,0.000000,0.000000,0.200714,0.000000,0.000000,0.000000,0.000000,1.916532,0.045000,UT
7,North Carolina,0.4,0.400000,10000.000000,0.029167,0.104364,0.104717,0.077600,0.481852,0.090000,2.488881,0.431758,0.105432,0.152131,657.983800,3.658617,1.181583,2.964933,22.475436,1.160476,0.052500,0.185670,0.154026,0.276667,76.885179,0.431087,3.241209,1.360000,2.175098,9.553522,0.173529,6.432026,0.082667,6.718313,0.0000

In [64]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
engagement_state.iloc[:,38:72] = min_max_scaler.fit_transform(engagement_state.iloc[:,38:72])

engagement_state.describe()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,pct_free/reduced,pp_total_raw,eng_ind_13945,eng_ind_18982,eng_ind_20619,eng_ind_21917,eng_ind_22767,eng_ind_23927,eng_ind_26696,eng_ind_30008,eng_ind_32032,eng_ind_32050,eng_ind_33185,eng_ind_47267,eng_ind_48333,eng_ind_49094,eng_ind_50282,eng_ind_50626,eng_ind_50692,eng_ind_52448,eng_ind_55032,eng_ind_59304,eng_ind_61945,eng_ind_64850,eng_ind_65277,eng_ind_69706,eng_ind_72703,eng_ind_73323,eng_ind_78662,eng_ind_86703,eng_ind_93376,eng_ind_94218,eng_ind_95767,eng_ind_95859,eng_ind_98845,eng_ind_99580,pct_acc_13945,pct_acc_18982,pct_acc_20619,pct_acc_21917,pct_acc_22767,pct_acc_23927,pct_acc_26696,pct_acc_30008,pct_acc_32032,pct_acc_32050,pct_acc_33185,pct_acc_47267,pct_acc_48333,pct_acc_49094,pct_acc_50282,pct_acc_50626,pct_acc_50692,pct_acc_52448,pct_acc_55032,pct_acc_59304,pct_acc_61945,pct_acc_64850,pct_acc_65277,pct_acc_69706,pct_acc_72703,pct_acc_73323,pct_acc_78662,pct_acc_86703,pct_acc_93376,pct_acc_94218,pct_acc_95767,pct_acc_95859,pct_acc_98845,pct_acc_99580
count,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000,176.000000
mean,0.438121,13543.817527,2.753774,2.129820,2.351644,2.567462,2.142500,6.482855,8.962540,4.963765,0.642130,6.290973,1501.820479,13.915920,131.277593,8.175653,11.487763,2.831132,2.137815,2.042687,1.713402,2.422549,708.483836,4.690775,9.370485,14.722088,21.305991,24.766004,1.920616,38.180260,0.671444,6.526279,0.810584,16.599251,2996.906414,25.870332,0.039403,0.129920,0.059312,0.109189,0.090625,0.057348,0.055098,0.070767,0.049004,0.022985,0.205196,0.143846,0.076046,0.054014,0.093255,0.047894,0.043351,0.034544,0.038991,0.102185,0.120449,0.067072,0.183356,0.015657,0.037132,0.047291,0.102469,0.118899,0.015541,0.030911,0.064644,0.149309,0.189051,0.065487
std,0.194758,3751.377661,17.558640,3.507152,4.946297,3.605867,3.779017,17.606824,19.396614,12.429066,2.434260,14.426204,1375.699274,19.694979,294.170377,15.669817,22.379426,10.532118,5.118863,8.522856,2.969815,3.236241,1016.878909,10.501554,13.367117,120.105085,55.166785,56.340906,2.856814,43.140586,3.186074,17.579537,2.374620,19.527961,4853.689375,53.830287,0.118971,0.207469,0.119299,0.173691,0.131783,0.121734,0.103297,0.125017,0.145905,0.078945,0.148102,0.186832,0.125450,0.099603,0.137485,0.120404,0.118976,0.101952,0.094941,0.123485,0.175013,0.133363,0.157685,0.092084,0.100505,0.107392,0.156282,0.135847,0.086322,0.089308,0.135123,0.186121,0.225096,0.134889
min,0.200000,6000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.445517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.387963,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,10000.000000,0.000000,0.295481,0.322232,0.416332,0.398947,0.285107,1.735331,0.586779,0.000000,1.360425,604.524546,3.724967,10.081029,1.330521,1.580085,0.280500,0.000000,0.283531,0.248800,0.638063,47.282358,0.554261,2.436245,0.000000,2.505000,0.422237,0.432430,12.688

In [ ]:
from sklearn.linear_model import Lasso




clf = linear_model.Lasso(alpha=0.1)
>>> clf.fit([[0,0], [1, 1], [2, 2]], [0, 1, 2])